### IFN680 Sokoban Assignment


The functions and classes defined in this module will be called by a marker script. 
You should complete the functions and classes according to their specified interfaces.

You are not allowed to change the defined interfaces.
That is, changing the formal parameters of a function will break the 
interface and triggers to a fail for the test of your code.
 

In [1]:
# Imports

import zipfile
import os

import search
import sokoban 
from sokoban import Warehouse

from collections import deque

In [27]:
def taboo_cells(warehouse):
    '''  
    Identify the taboo cells of a warehouse. A cell is called 'taboo' 
    if whenever a box get pushed on such a cell then the puzzle becomes unsolvable.  
    When determining the taboo cells, you must ignore all the existing boxes, 
    simply consider the walls and the target  cells.  
    Use only the following two rules to determine the taboo cells;
     Rule 1: if a cell is a corner and not a target, then it is a taboo cell.
     Rule 2: all the cells between two corners along a wall are taboo if none of 
             these cells is a target.
    
    @param warehouse: a Warehouse object

    @return
       A string representing the puzzle with only the wall cells marked with 
       an '#' and the taboo cells marked with an 'X'.  
       The returned string should NOT have marks for the worker, the targets,
       and the boxes.  
    '''   
    
    taboo_string = ""
    # list to store cells (x,y) for rule 1
    corner_taboo_cells = []
    # list to store cells (x,y) for rule 2
    other_taboo_cells = []
    # list to store cells (x,y) of targets
    targetsXY = []
    
    cells = str(warehouse).split('\n')
    
    index = 0
    for strings in cells:
        cells[index] = list(strings)
        index += 1
    
    # Codes to get cells for rule 1 
    # Copy cells into temp by splitting everything
    temp = cells[:]
    
    for r_indx, row in enumerate(cells):
        inside_cell = False    
        for c_indx, char in enumerate(row):
            
            # Always turn the goal into a legit spot
            if char == "." or char == '$' or char == '*':
                cells[r_indx][c_indx] = " " # TODO change this later
                inside_cell = True
            
            if char == "." or char == "*":
                targetsXY.append((r_indx, c_indx))
                    
                
            # If a wall is encountered, check if it is within the playing area
            elif char == '#':
                inside_cell = True
                for index in range(c_indx):
                    if cells[r_indx][index] == '#':
                        inside_cell = False
                cells[r_indx][c_indx] = char
                
            else:
                # Check if the space is within the cell and if it's in the corner of the warehouse
                if inside_cell == False or r_indx == 0 or r_indx == len(cells) - 1 or c_indx == 0 or c_indx == len(cells[r_indx]) - 1:
                    cells[r_indx][c_indx] = char
                else:
                    
                    taboo1 = temp[r_indx][c_indx - 1] == '#' and temp[r_indx - 1][c_indx] == '#' # Checks left top
                    taboo2 = temp[r_indx][c_indx + 1] == '#' and temp[r_indx - 1][c_indx] == '#' # Checks right top
                    taboo3 = temp[r_indx][c_indx - 1] == '#' and temp[r_indx + 1][c_indx] == '#' # Checks left bottom
                    taboo4 = temp[r_indx][c_indx + 1] == '#' and temp[r_indx + 1][c_indx] == '#' # Checks right bottom
                    
                    if taboo1 or taboo2 or taboo3 or taboo4:
                        cells[r_indx][c_indx] = 'X'
                        corner_taboo_cells.append((r_indx, c_indx))
                    else:
                        cells[r_indx][c_indx] = ' '
    
    # Codes to get cells for Rule 2
    potentialTabooXY = []
    isTaboo = False
    
    for aCell in corner_taboo_cells:
        # if a corner taboo cell is in a Top Left Corner
        if cells[aCell[0]][aCell[1] - 1] == '#' and cells[aCell[0] - 1][aCell[1]] == '#':
            # Check and get the all the cells below the corner taboo cell untill a wall is encountered
            y = aCell[0] + 1
            
            while cells[y][aCell[1]] != '#':
                potentialTabooXY.append((y, aCell[1]))
                y = y + 1
            
            if len(potentialTabooXY) > 0:
                # Check all the cells are in between two corners
                if potentialTabooXY[-1] in corner_taboo_cells:
                    isTaboo = True
                    for aXY in potentialTabooXY:
                        # Check the left side of all the cells, if one of is not a wall,
                        # then all cells are not taboo cells
                        if cells[aXY[0]][aXY[1]- 1 ] != '#':
                            isTaboo = False
                        
                        # Check if any of the cells is a target, if it is a target.
                        # then all cells are not taboo cells
                        if aXY in targetsXY:
                            isTaboo = False
            
            if isTaboo:
                other_taboo_cells.extend(potentialTabooXY)
                isTaboo = False
            
            potentialTabooXY = []
                
            # Check and get the all the cells to the right the corner taboo cell untill a wall is encountered
            x = aCell[1] + 1
            while cells[aCell[0]][x] != '#':
                potentialTabooXY.append((aCell[0], x))
                x = x + 1
                
            if len(potentialTabooXY) > 0:    
                # Check all the cells are in between two corners
                if potentialTabooXY[-1] in corner_taboo_cells:
                    isTaboo = True
                    for aXY in potentialTabooXY:
                        # Check the top side of all the cells, if one of is not a wall,
                        # then all cells are not taboo cells
                        if cells[aXY[0] - 1][aXY[1]] != '#':
                            isTaboo = False
                            
                        # Check if any of the cells is a target, if it is a target,
                        # then all cells are not taboo cells    
                        if aXY in targetsXY:
                            isTaboo = False
            
            if isTaboo:
                other_taboo_cells.extend(potentialTabooXY)
                isTaboo = False
            
            potentialTabooXY = []
                
        # if a corner taboo cell is in a Top Right corner
        # This part of the code is similar to the section above, the only difference is
        # this will check the cells along the walls of the top right corner
        if cells[aCell[0]][aCell[1] + 1] == '#' and cells[aCell[0] - 1][aCell[1]] == '#':
            y = aCell[0] + 1
            while cells[y][aCell[1]] != '#':
                potentialTabooXY.append((y, aCell[1]))
                y = y + 1
            
            if len(potentialTabooXY) > 0:
                if potentialTabooXY[-1] in corner_taboo_cells:
                    isTaboo = True
                    for aXY in potentialTabooXY:
                        if cells[aXY[0]][aXY[1] + 1 ] != '#':
                            isTaboo = False
                        if aXY in targetsXY:
                            isTaboo = False
            
            if isTaboo:
                other_taboo_cells.extend(potentialTabooXY)
                isTaboo = False
            
            potentialTabooXY = []
                
            x = aCell[1] - 1
            while cells[aCell[0]][x] != '#':
                potentialTabooXY.append((aCell[0], x))
                x = x - 1
            
            if len(potentialTabooXY) > 0:
                if potentialTabooXY[-1] in corner_taboo_cells:
                    isTaboo = True
                    for aXY in potentialTabooXY:
                        # Check top
                        if cells[aXY[0] - 1][aXY[1]] != '#':
                            isTaboo = False
                        if aXY in targetsXY:
                            isTaboo = False
            
            if isTaboo:
                other_taboo_cells.extend(potentialTabooXY)
                isTaboo = False
            
            potentialTabooXY = []
            
        # if a corner taboo cell is in a Bottom Left Corner
        # This part of the code is similar to the section above, the only difference is
        # this will check the cells along the walls of the Bottom Left Corner
        if cells[aCell[0]][aCell[1] - 1] == '#' and cells[aCell[0] + 1][aCell[1]] == '#':
            y = aCell[0] - 1
            
            while cells[y][aCell[1]] != '#':
                potentialTabooXY.append((y, aCell[1]))
                y = y - 1
            
            if len(potentialTabooXY) > 0:
                if potentialTabooXY[-1] in corner_taboo_cells:
                    isTaboo = True
                    for aXY in potentialTabooXY:
                        if cells[aXY[0]][aXY[1] - 1 ] != '#':
                            isTaboo = False
                        if aXY in targetsXY:
                            isTaboo = False
            
            if isTaboo:
                other_taboo_cells.extend(potentialTabooXY)
                isTaboo = False
            
            potentialTabooXY = []
                
            x = aCell[1] + 1
            
            while cells[aCell[0]][x] != '#':
                potentialTabooXY.append((aCell[0], x))
                x = x + 1
            
            if len(potentialTabooXY) > 0:
                if potentialTabooXY[-1] in corner_taboo_cells:
                    isTaboo = True
                    for aXY in potentialTabooXY:
                        if cells[aXY[0] + 1][aXY[1]] != '#':
                            isTaboo = False
                        if aXY in targetsXY:
                            isTaboo = False
            
            if isTaboo:
                other_taboo_cells.extend(potentialTabooXY)
                isTaboo = False
            
            potentialTabooXY = []
            
        # if a corner taboo cell is in a Bottom Right Corner
        # This part of the code is similar to the section above, the only difference is
        # this will check the cells along the walls of Bottom Right Corner
        if cells[aCell[0]][aCell[1] + 1] == '#' and cells[aCell[0] + 1][aCell[1]] == '#':
            y = aCell[0] - 1
            
            while cells[y][aCell[1]] != '#':
                potentialTabooXY.append((y, aCell[1]))
                y = y - 1
            
            if len(potentialTabooXY) > 0:
                if potentialTabooXY[-1] in corner_taboo_cells:
                    isTaboo = True
                    for aXY in potentialTabooXY:
                        if cells[aXY[0]][aXY[1] + 1 ] != '#':
                            isTaboo = False
                        if aXY in targetsXY:
                            isTaboo = False
            
            if isTaboo:
                other_taboo_cells.extend(potentialTabooXY)
                isTaboo = False
            
            potentialTabooXY = []
            
            x = aCell[1] - 1
            while cells[aCell[0]][x] != '#':
                potentialTabooXY.append((aCell[0], x))
                x = x - 1
            
            if len(potentialTabooXY) > 0:
                if potentialTabooXY[-1] in corner_taboo_cells:
                    isTaboo = True
                    for aXY in potentialTabooXY:
                        if cells[aXY[0] + 1][aXY[1]] != '#':
                            isTaboo = False
                        if aXY in targetsXY:
                            isTaboo = False
            
            if isTaboo:
                other_taboo_cells.extend(potentialTabooXY)
                isTaboo = False
            
            potentialTabooXY = []
            
    
    for aCell in other_taboo_cells:
        cells[aCell[0]][aCell[1]] = 'X'                                        
                                
    cells = cells[0:]
             
    for i, row in enumerate(cells):
        taboo_string += ''.join(row)
        if i < len(cells) - 1:  # Don't add a newline after the last row
            taboo_string += "\n"  

    return taboo_string

In [23]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def find_size(warehouse):
    '''
    Helper function in order to find the size of the warehouse
    Both x and y axis

    @param: warehouse: the warehouse object
    
    @return
            x_size:  the column size of warehouse
            y_size: the row size of warehouse
    '''
    
    # Determine the y size of the warehouse by splitting all '\n'
    cells = str(warehouse).split('\n')    
    
    # Split all the strings into individual characters in order to determine 
    # the x size
    index = 0
    for strings in cells:
        cells[index] = list(strings)
        index += 1

    # Add +1 because lists starts with 0
    x_size = len(cells[0])
    y_size = len(cells)

    return (x_size, y_size)

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def find_action(goal_node):
    '''
    Helper function to find the list of action from node list if the possible solution is
    found to fix the Sokoban Puzzle

    @param goal_node: the list of nodes has reached the goal test
    
    @return
            the list of actions from every node 
    '''
    path = goal_node.path()
    step_move = []
    for node in path:
        step_move.append(node.action)
    return step_move[1:]

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def is_move_possible(warehouse, action):
    '''
    Helper function to see if the next move is possible with particular action
    
    @param warehouse: the warehouse object
    @param action: the type of action. For example :['Left','Right','Up','Down']
    
    @return
        True: if the move is possible
        False: vice versa
    '''
    worker_x = warehouse.worker[0]
    worker_y = warehouse.worker[1]
    
    boxes = warehouse.boxes
    walls = warehouse.walls
    
    # Determine what kind of action it is and set the appropriate
    # number to x and y
    # xx and yy is to check if there is anything behind/infront the worker destionation
    x = 0 
    y = 0
    xx = 0
    yy = 0
    if (action == "Up"):
        y  -= 1
        yy -= 2
    elif (action == "Down"):
        y  += 1
        yy += 2
    elif (action == "Left"):
        x  -= 1
        xx -= 2
    elif (action == "Right"):
        x  += 1
        xx += 2
    else:
        return False
    
    # For every action
    # We check if the worker's new coordinates is inside the wall
    # or if the worker is moving to a box where there is another box ehind
    # or if the worker is pushing the box into a wall
    if (worker_x + x, worker_y + y) in walls:
        return False
    elif (worker_x + x, worker_y + y) in boxes and (worker_x + xx, worker_y + yy) in boxes:
        return False
    elif (worker_x + x, worker_y + y) in boxes and (worker_x + xx, worker_y + yy) in walls:
        return False
    else:
        return True
        
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def find_move(action):
    '''
    Helper function to find the position of x and y with particular action
    
    @param action: the type of action. For example :['Left','Right','Up','Down']
    
    @return
            x : x value  should be added one. For example, if the action is Right, 
            y : y value should be added one. For example, if the action is Down
    '''
    x = 0
    y = 0
    if (action == "Up"):
        y -= 1
    elif (action == "Down"):
        y += 1
    elif (action == "Left"):
        x -= 1
    elif (action == "Right"):
        x += 1
    
    return x, y

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def is_taboo(x, y, walls):
    '''
    Helper function to check if the corner walls are taboo
    @param x: the x coordinates
    @param y: the y coordinates
    @param walls: the locations of the walls
    
    @return
        True: if the coordinates is taboo
        False: vice versa
    '''
    taboo_1 = (x, y - 1) in walls and (x - 1, y) in walls # Top Left
    taboo_2 = (x, y - 1) in walls and (x + 1, y) in walls # Top Right
    taboo_3 = (x, y + 1) in walls and (x - 1, y) in walls # Bottom Left
    taboo_4 = (x, y + 1) in walls and (x + 1, y) in walls # Botoom Right
    if taboo_1 or taboo_2 or taboo_3 or taboo_4:
        return True
    else:
        return False

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def is_taboo_cells(unit_x,unit_y,taboo_cells):
    '''
    Helper function to check whether the position of moving box is located within the place market with 'X'
    @param unit_x: x coordinate of moving box
    @param unit_y: y coordinate of moving box
    @param taboo_cells: the list of string of taboo generated by taboo cells function
    @return:
            True: if the position is located in the place marked with 'X'
            False: vice versa
    '''
    taboo_rc = taboo_cells.split("\n")
    index = 0
    for strings in taboo_rc:
        taboo_rc[index] = list(strings)
        index +=1
    
    taboo_rc = taboo_rc[1:]
    
    if taboo_rc[unit_y-1][unit_x] == 'X':
        return True
    else:
        return False

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -   
   
def isNot_walls_next_move(x,y,walls):
    '''
    The function to check the location of walls

    @param x : coordinate x of unit (worker)
    @param y : cooridnate y of unit (worker)
            walls : the coordinates of walls in the updated warehouse
    @return
            True: if the walls is located at the same position as worker
            False: vice versa
    '''
    if (x,y) in walls:
        return False
    else:
        return True
        
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def isNot_boxes_next_move(x,y,boxes):
    '''
    The function to check the location of boxes

    @param x : coordinate x of unit (worker)
    @param y : cooridnate y of unit (worker)
            boxes : the coordinates of boxes in the updated warehouse
    @return
            True: if the boxes is located at the same position as worker
            False: vice versa
    '''
    if (x,y) in boxes:
        return False    
    else:
        return True
  
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  
  
def manhattan(coor_1, coor_2):
    '''
    The helper function to calculate the distance between player and boxes,
    the distance between player and targets

    @param coor_1: the x coordinate of boxes or targets
    @param coor_2: the y coordinate of boxes or targets

    @return: the distance between player and boxes or player and targets
    '''
    return abs(coor_1[0] - coor_2[0]) + abs(coor_1[1] - coor_2[1])

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def get_Distance(start, locations):
    '''
    The helper function to find the shortest distance between player
    and boxes or player and targets

    @param start: the coordinate of starting location
    @param locations: the coordinate of destination

    @return: the shortest distance 
    '''
    min_dist = 100000
    
    for coor in locations:
        dist = manhattan(start, coor)
        if (dist < min_dist):
            min_dist = dist
            
    return min_dist

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def get_Heuristic(node):
    '''
    The function used to find or get heuristic value for each node

    @param node: the changed state of warehouse object

    @return: the heuristic value for node to reach the goal
    '''
    warehouse = node.state
    worker = warehouse.worker
    boxes = warehouse.boxes
    targets = warehouse.targets
    
    sum = 0
    
    playerMin = get_Distance(worker, boxes)
    sum += playerMin
    
    for box in boxes:
        boxMin = get_Distance(box, targets)
        sum += boxMin
        
    return sum


In [2]:
def my_team():
    '''
    Return the list of the team members of this assignment submission as a list
    of triplet of the form (student_number, first_name, last_name)
    
    '''
    return ('N11407697', 'Vishnu', 'Shaji')
#    raise NotImplementedError()

In [3]:
# Function to check the corners.
def is_corner(cell, warehouse):
    '''Check if the given cell is a corner inside the warehouse'''
    x, y = cell
    walls = warehouse.walls
    # Check the four possible corner configurations
    top_left = (x, y-1) in walls and (x-1, y) in walls
    top_right = (x, y-1) in walls and (x+1, y) in walls
    bottom_left = (x, y+1) in walls and (x-1, y) in walls
    bottom_right = (x, y+1) in walls and (x+1, y) in walls
    return top_left or top_right or bottom_left or bottom_right
# Function to get the valid cells
def cells_between_corners(cell1, cell2):
    '''Get the cells between two corners along a wall'''
    x1, y1 = cell1
    x2, y2 = cell2
    # If the two corners are horizontal to each other
    if x1 == x2:
        return [(x1, y) for y in range(min(y1, y2)+1, max(y1, y2))]
    # If the two corners are vertical to each other
    elif y1 == y2:
        return [(x, y1) for x in range(min(x1, x2)+1, max(x1, x2))]
    else:
        return []



def taboo_cells(warehouse):
    '''  
    Identify the taboo cells of a warehouse. A cell inside a warehouse is 
    called 'taboo' if whenever a box get pushed on such a cell then the puzzle 
    becomes unsolvable.  
    When determining the taboo cells, you must ignore all the existing boxes, 
    simply consider the walls and the target cells.  
    Use only the following two rules to determine the taboo cells;
     Rule 1: if a cell is a corner inside the warehouse and not a target, 
             then it is a taboo cell.
     Rule 2: all the cells between two corners inside the warehouse along a 
             wall are taboo if none of these cells is a target.
    
    @param warehouse: a Warehouse object

    @return
       A string representing the puzzle with only the wall cells marked with 
       an '#' and the taboo cells marked with an 'X'.  
       The returned string should NOT have marks for the worker, the targets,
       and the boxes.  
    '''
    '''Identify the taboo cells of a warehouse'''
    taboo = set()
    
    # Determine the number of columns and rows
    ncols = max(x for x, _ in warehouse.walls) + 1
    nrows = max(y for _, y in warehouse.walls) + 1
    
    for x in range(ncols):
        for y in range(nrows):
            cell = (x, y)
            # Rule 1: Check for corner cells which are not targets
            if is_corner(cell, warehouse) and cell not in warehouse.targets:
                taboo.add(cell)
            # Rule 2: Check for cells between two corners along a wall
            for dx, dy in [(0, 1), (1, 0)]:
                neighbour = (x + dx, y + dy)
                if is_corner(neighbour, warehouse) and cell not in warehouse.targets and neighbour not in warehouse.targets:
                    for c in cells_between_corners(cell, neighbour):
                        taboo.add(c)
                        
    # Create a representation of the warehouse with taboo cells marked as 'X'
    result = []
    for y in range(nrows):
        row = ""
        for x in range(ncols):
            if (x, y) in warehouse.walls:
                row += '#'
            elif (x, y) in taboo:
                row += 'X'
            else:
                row += ' '
        result.append(row)
    #return taboo
    return '\n'.join(result)
    #raise NotImplementedError()


In [28]:
class SokobanPuzzle(search.Problem):
    '''
    An instance of the class 'SokobanPuzzle' represents a Sokoban puzzle.
    An instance contains information about the walls, the targets, the boxes
    and the worker.

    Your implementation should be fully compatible with the search functions of 
    the provided module 'search.py'. 
    
    Each instance should have at least the following attributes
    - self.allow_taboo_push
    - self.macro
    
    When self.allow_taboo_push is set to True, the 'actions' function should 
    return all possible legal moves including those that move a box on a taboo 
    cell. If self.allow_taboo_push is set to False, those moves should not be
    included in the returned list of actions.
    
    If self.macro is set True, the 'actions' function should return 
    macro actions. If self.macro is set False, the 'actions' function should 
    return elementary actions.
    
    
    '''
    #
    #         "INSERT YOUR CODE HERE"
    #
    #     Revisit the sliding puzzle and the pancake puzzle for inspiration!
    #
    #     Note that you will need to add several functions to 
    #     complete this class. For example, a 'result' function is needed
    #     to satisfy the interface of 'search.Problem'.
    
    def __init__(self, warehouse, allow_taboo_push=True, macro=False):
        """Initialize the Sokoban puzzle."""
        self.initial = (warehouse.worker, tuple(warehouse.boxes))
        self.allow_taboo_push = allow_taboo_push
        self.macro = macro
        self.walls = warehouse.walls
        self.targets = warehouse.targets
        self.taboo_cells = taboo_cells(warehouse)
        
        # Goal test would often be to check if all boxes are on targets
        self.goal = set(warehouse.targets)
        #raise NotImplementedError()

    def actions(self, state):
        worker_pos, boxes = state
        """
        Return the list of actions that can be executed in the given state 
        if these actions do not push a box in a taboo cell.
        The actions must belong to the list ['Left', 'Down', 'Right', 'Up']

        @param state: a changed state of valid warehouse object
        """
        move = []
        worker_x, worker_y = worker_pos
        walls = self.walls
        
        ## Check the action is not pushing the boxes to the taboo cells, and also the box shouldn't move if there's box or walls located behind them
        ## in the same moving direction

        #Left
        if isNot_walls_next_move(worker_x-1,worker_y,walls):
            if isNot_boxes_next_move(worker_x-1,worker_y,boxes):
                move.append("Left")
            else:
                if isNot_boxes_next_move(worker_x-2,worker_y,boxes) and isNot_walls_next_move(worker_x-2,worker_y,walls) and \
                    is_taboo_cells(worker_x-2,worker_y,self.taboo_cells) == False :
                    move.append("Left")
        #Right
        if isNot_walls_next_move(worker_x+1,worker_y,walls):
            if isNot_boxes_next_move(worker_x+1,worker_y,boxes):
                move.append("Right")
            else:
                if isNot_boxes_next_move(worker_x+2,worker_y,boxes) and isNot_walls_next_move(worker_x+2,worker_y,walls) and \
                    is_taboo_cells(worker_x+2,worker_y,self.taboo_cells) == False :
                    move.append("Right")
        #Up
        if isNot_walls_next_move(worker_x,worker_y-1,walls):
            if isNot_boxes_next_move(worker_x,worker_y-1,boxes):
                move.append("Up")
            else:
                if isNot_boxes_next_move(worker_x,worker_y-2,boxes) and isNot_walls_next_move(worker_x,worker_y-2,walls) and \
                    is_taboo_cells(worker_x,worker_y-2,self.taboo_cells) == False :
                    move.append("Up")
        #Down
        if isNot_walls_next_move(worker_x,worker_y+1,walls):
            if isNot_boxes_next_move(worker_x,worker_y+1,boxes):
                move.append("Down")
            else:
                if isNot_boxes_next_move(worker_x,worker_y+2,boxes) and isNot_walls_next_move(worker_x,worker_y+2,walls) and \
                    is_taboo_cells(worker_x,worker_y+2,self.taboo_cells) == False :
                    move.append("Down")
                    
        return move  


    
    def result(self, state, action):
        '''
        Function that applies an action to the a state (warehouse) and return
        a new state.
        
        @param state: A Warehouse object
        
        @param action: An action from list ['Left', 'Down', 'Right', 'Up'] 
            
        @return
            Return the state (a Warehouse with the updated location of worker 
            and boxes) after applying an action.
        
        '''
        worker_pos, boxes = state
        worker = list(worker_pos)
        boxes = list(boxes)
        
        assert action in self.actions(state)
                
        x, y = find_move(action)
        worker[0] += x
        worker[1] += y
              
        for idx, box in enumerate(boxes):
            if box == (worker[0], worker[1]):
                box_x = box[0] + x
                box_y = box[1] + y
                boxes[idx] = (box_x, box_y)
        
        return (tuple(worker), tuple(boxes))

    
    def goal_test(self, state):
        """Return True if all boxes are on target positions."""
        _, boxes = state
        return all(box in self.targets for box in boxes)
    
    def h(self, node):
        """Heuristic for the Sokoban problem: sum of Manhattan distances from boxes to closest targets."""
        worker, boxes = node.state
        total_distance = 0
        for box in boxes:
            distances = [abs(box[0] - target[0]) + abs(box[1] - target[1]) for target in self.targets]
            total_distance += min(distances)
        return total_distance

        #raise NotImplementedError


In [6]:
# Testing the refactored SokobanPuzzle class
#wh = Warehouse()
#warehouse_str = "./warehouses/warehouse_01.txt"

#wh.load_warehouse(warehouse_str)
#puzzle = SokobanPuzzle(wh)
#actions = puzzle.actions(wh)
#actions


In [7]:
# Testing the result method using an instance of the SokobanPuzzle class
#current_state = puzzle.initial
#test_actions = ["Up"]

#for action in test_actions:
    #current_state = puzzle.result(current_state, action)

# Print the resulting warehouse state
#print(current_state)

In [8]:
def taboo_string_to_coords(taboo_str):
    taboo_set = set()
    for y, row in enumerate(taboo_str.splitlines()):
        for x, cell in enumerate(row):
            if cell == 'X':
                taboo_set.add((x, y))
    return taboo_set

In [9]:
def check_action_seq(warehouse, action_seq):
    '''
    Determine if the sequence of actions listed in 'action_seq' is legal or not.
    
    @param warehouse: a valid Warehouse object
    @param action_seq: a sequence of legal actions.
           For example, ['Left', 'Down', Down','Right', 'Up', 'Down']
    @return: A string representing the state of the puzzle after applying
             the sequence of actions or 'Failure'.
    '''
    
    # Convert the warehouse to a SokobanPuzzle instance
    puzzle = SokobanPuzzle(warehouse)
    worker_position, boxes = puzzle.initial
    taboo_set = taboo_string_to_coords(puzzle.taboo_cells)
    
    state = (worker_position, tuple(boxes))
    
    for action in action_seq:
        # Check if the action is valid for the current state
        if action not in puzzle.actions(state):
            return "Failure"
        # Apply the action to get the new state
        state = puzzle.result(state, action)
        _, boxes = state
        # If a box is pushed into a taboo cell, the sequence is invalid
        if any(box in taboo_set for box in boxes):
            return "Failure"

    # Convert the final state to warehouse representation and return it
    wh = Warehouse()
    wh.worker = state[0]
    wh.boxes = list(state[1])
    wh.walls = warehouse.walls  # Copy the walls from the original warehouse
    wh.targets = warehouse.targets  # Copy the targets from the original warehouse
    return str(wh)


# Update the SokobanPuzzle class with the refined check_action_seq method
setattr(SokobanPuzzle, "check_action_seq", check_action_seq)

In [29]:
def manhattan_distance(point1, point2):
    """Return the Manhattan distance between two points."""
    return abs(point1[0] - point2[0]) + abs(point1[1] - point2[1])


def sokoban_heuristic(node, problem):
    """Heuristic function for the Sokoban puzzle."""
    state = node.state
    _, boxes = state
    targets = problem.warehouse.targets
    return sum(min(manhattan_distance(box, target) for target in targets) for box in boxes)



def solve_sokoban_elem(warehouse):
    '''    
    This function should solve using elementary actions 
    the puzzle defined in a file.
    
    @param warehouse: a valid Warehouse object

    @return
        If puzzle cannot be solved return the string 'Impossible'
        If a solution was found, return a list of elementary actions that solves
            the given puzzle coded with 'Left', 'Right', 'Up', 'Down'
            For example, ['Left', 'Down', Down','Right', 'Up', 'Down']
            If the puzzle is already in a goal state, simply return []
    '''
    
    ##         "INSERT YOUR CODE HERE"
    
    puzzle = SokobanPuzzle(warehouse)
    
    # Check if the initial state is already a goal state
    if puzzle.goal_test(puzzle.initial):
        return []
    
    solution = search.astar_graph_search(puzzle)


    if solution:
        return solution.solution()
    return ['Impossible']

    
   # raise NotImplementedError()


In [11]:
def can_go_there(warehouse, dst):
    '''    
    Determine whether the worker can walk to the cell dst=(row,column) 
    without pushing any box.
    '''
    
    # Define the BFS algorithm
    def bfs(grid, start):
        visited = set()
        queue = [start]
        
        while queue:
            x, y = queue.pop(0)
            if (x, y) in visited:
                continue
            visited.add((x, y))
            
            # Check the four adjacent cells (left, right, up, down)
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                next_x, next_y = x + dx, y + dy
                
                # Make sure it's within boundaries and not a wall or box
                if 0 <= next_x < len(grid) and 0 <= next_y < len(grid[0]) and \
                   grid[next_x][next_y] not in ['#', '$']:
                    queue.append((next_x, next_y))
        return visited
    
    # Convert the warehouse to a grid representation
    grid = str(warehouse).splitlines()
    
    # Get all reachable cells from the worker's starting position using BFS
    reachable_cells = bfs(grid, warehouse.worker)
    
    # Return True if dst is in the set of reachable cells
    return dst in reachable_cells

In [12]:
def bfs_pathfinding(start, goal, walls, boxes):
    """Find a path from start to goal avoiding walls and boxes."""
    frontier = [start]
    explored = set()
    came_from = {start: None}  # To trace the path
    
    print(f"Starting BFS from {start} to {goal}")  # <-- Add this print statement


    while frontier:        
        current = frontier.pop(0)
        if current == goal:
            # Reconstruct the path
            path = []
            while current != start:
                previous = came_from[current]
                dx, dy = current[0] - previous[0], current[1] - previous[1]
                if dx == 1:
                    path.append("Right")
                elif dx == -1:
                    path.append("Left")
                elif dy == 1:
                    path.append("Down")
                elif dy == -1:
                    path.append("Up")
                current = previous
            return path[::-1]  # Reverse the path

        explored.add(current)
        
        print(f"Current node: {current}")

        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            x, y = current[0] + dx, current[1] + dy
            next_node = (x, y)
            
            print(f"Evaluating next node: {next_node}")

            if next_node in walls or next_node in boxes or next_node in explored or next_node in frontier:
                
                print(f"Skipping next node: {next_node}")

                continue
            print(f"Adding next node to frontier: {next_node}")
            frontier.append(next_node)
            came_from[next_node] = current
            
        print(f"Frontier at end of BFS: {frontier}")  # <-- Add this print statement
        print(f"Explored nodes at end of BFS: {explored}")  # <-- Add this print statement
        print(f"Came_from at end of BFS: {came_from}")  # <-- Add this print statement


    return None  # No path found


In [13]:
# HELPER FUNCTIONS TO SOLVE MACRO
def find_unmatched_box(boxes, targets):
    for box in boxes:
        if box not in targets:
            return box
    return None

def closest_target_for_box(box, targets):
    return min(targets, key=lambda target: manhattan_distance(box, target))

def path_for_worker_to_box(warehouse, box, target):
    # We need to determine which side of the box the worker should approach from
    # For simplicity, let's assume the worker will push the box in a straight line towards the target
    # Therefore, the worker should approach the box from the opposite direction

    print(f"Box at {box} should be moved towards {target}")

    dx, dy = box[0] - target[0], box[1] - target[1]
    if dx > 0:
        worker_dst = (box[0] + 1, box[1])
    elif dx < 0:
        worker_dst = (box[0] - 1, box[1])
    elif dy > 0:
        worker_dst = (box[0], box[1] + 1)
    else:
        worker_dst = (box[0], box[1] - 1)

    print(f"Desired worker destination: {worker_dst}")
    
    # If worker is already at the desired destination, return an empty path
    if warehouse.worker == worker_dst:
        return []
    
    # Now use BFS or any pathfinding algorithm to find the path for the worker to this destination
    return bfs_pathfinding(warehouse.worker, worker_dst, warehouse.walls, warehouse.boxes)
    # Ensure the path doesn't involve pushing any boxes
    # If a path is found, return it. Otherwise, return None.

def push_direction_for_box(box, target):
    if box[0] < target[0]:
        return 'Down'
    elif box[0] > target[0]:
        return 'Up'
    elif box[1] < target[1]:
        return 'Right'
    else:
        return 'Left'

def is_push_feasible(puzzle, box, direction):
    """Check if the push in the given direction is feasible."""
    dx, dy = 0, 0
    
    # Determine the move based on direction
    if direction == "Up":
        dx, dy = -1, 0
    elif direction == "Down":
        dx, dy = 1, 0
    elif direction == "Left":
        dx, dy = 0, -1
    elif direction == "Right":
        dx, dy = 0, 1

    new_box_position = (box[0] + dx, box[1] + dy)

    # Check for walls or other boxes in the new position
    if new_box_position in puzzle.walls or new_box_position in puzzle.initial[1]:
        return False

    return True

In [26]:
def solve_sokoban_macro(warehouse):
    solution = solve_sokoban_elem(warehouse)
    print(f"Initial worker position in warehouse: {warehouse.worker}")
    print(f"Initial boxes positions in warehouse: {warehouse.boxes}")
    curr_warehouse = warehouse
    last_x,last_y = curr_warehouse.worker
    total_macro_move = []
    
    if "Impossible" in solution or solution is None :
        return total_macro_move.append('Impossible')
    else:
        for action in solution:
            last_x, last_y = curr_warehouse.worker
            move_x,move_y = find_move(action)
            worker_x, worker_y = curr_warehouse.worker
            boxes = curr_warehouse.boxes
            worker_x = worker_x + move_x
            worker_y = worker_y + move_y
            worker = (worker_x,worker_y)


            #Left
            if action == "Left":
                if not isNot_boxes_next_move(worker_x,worker_y,boxes):
                    for index in range(len(boxes)):
                        if boxes[index] == (worker_x,worker_y):
                            boxes[index] = (worker_x-1,worker_y)
                                
                    total_macro_move.append(((last_x,last_y),action))
            #Right        
            if action == "Right":
                if not isNot_boxes_next_move(worker_x,worker_y,boxes):
                    for index in range(len(boxes)):
                        if boxes[index] == (worker_x,worker_y):
                            boxes[index] = (worker_x+1,worker_y)
                                
                    total_macro_move.append(((last_x,last_y),action))
            #Down        
            if action == "Down":
                if not isNot_boxes_next_move(worker_x,worker_y,boxes):
                    for index in range(len(boxes)):
                        if boxes[index] == (worker_x,worker_y):
                            boxes[index] = (worker_x,worker_y+1)
                                
                    total_macro_move.append(((last_x,last_y),action))
            #Up        
            if action == "Up":
                if not isNot_boxes_next_move(worker_x,worker_y,boxes):
                    for index in range(len(boxes)):
                        if boxes[index] == (worker_x,worker_y):
                            boxes[index] = (worker_x,worker_y-1)
                                
                    total_macro_move.append(((last_x,last_y),action))

            curr_warehouse = curr_warehouse.copy((worker_x, worker_y), boxes)
            last_x,last_y = worker_x, worker_y
            
        return total_macro_move  

In [15]:
def apply_macro_action(warehouse, action):
    """Apply a macro action to the warehouse."""
    box_position, direction = action
    dx, dy = 0, 0
    
    if direction == "Up":
        dx, dy = -1, 0
    elif direction == "Down":
        dx, dy = 1, 0
    elif direction == "Left":
        dx, dy = 0, -1
    elif direction == "Right":
        dx, dy = 0, 1
    
    # Move the box
    new_box_position = (box_position[0] + dx, box_position[1] + dy)
    warehouse.boxes.remove(box_position)
    warehouse.boxes.append(new_box_position)
    
    # Update the worker's position to be where the box was before it was pushed
    warehouse.worker = box_position

def dfs_solve(warehouse, actions):
    print("Current warehouse state:")
    print(warehouse)
    
    # Base case: If all boxes are on targets, return the actions
    if set(warehouse.boxes) == set(warehouse.targets):
        return actions

    for box in warehouse.boxes:
        # Skip if box is already on a target
        if box in warehouse.targets:
            continue

        # Get the closest target for the box
        target = closest_target_for_box(box, warehouse.targets)
        
        print(f"Attempting to move box at {box} to target at {target}")

        # Find the worker's path to the box
        worker_path = path_for_worker_to_box(warehouse, box, target)
        print(f"Worker path to box: {worker_path}")
        if not worker_path:
            continue

        # Determine the direction to push the box towards the target
        direction = push_direction_for_box(box, target)
        new_action = (box, direction)

        # Clone the warehouse and apply the action
        new_warehouse = warehouse.copy()
        apply_macro_action(new_warehouse, new_action)

        # The worker should now be on the opposite side of the box. We need to reposition the worker to the original side of the box so it can continue pushing. This step was missing from the original code.
        worker_path = path_for_worker_to_box(new_warehouse, box, target)
        if not worker_path:
            continue

        # Recursively solve the remaining puzzle
        result = dfs_solve(new_warehouse, actions + [new_action])
        if result:
            return result

    return None

In [16]:
# Testing the initialization of macro action
#wh.load_warehouse("./warehouses/warehouse_05.txt")
#macro = solve_sokoban_macro(wh)
#macro